In [ ]:
%sql
/* 02_silver_encounters_with_patients_join.ipynb
SOURCE: kardia_silver.silver_encounters + kardia_silver.silver_patients
OUTPUT: kardia_silver.silver_encounters_with_patients
PATTERN: Snapshot left join to attach demographic details to each encounter
NOTE: GENDER and BIRTH_YEAR are already cleaned and validated upstream.
      This join runs on a small single-node cluster where broadcast is unnecessary;
      however, on larger clusters, BROADCAST(p) would avoid shuffle overhead.
TRIGGER: CTAS; rerun after each encounter load or on a fixed schedule */

-- 1. Ensure database exists
CREATE DATABASE IF NOT EXISTS kardia_silver;

-- 2. Create a snapshot table that attaches patient demographics to each encounter
CREATE OR REPLACE TABLE kardia_silver.silver_encounters_with_patients AS
SELECT  e.EncounterID        AS encounter_id,
        e.PatientID          AS patient_id,
        e.START_TS           AS start_ts,
        e.CODE               AS code,
        e.DESCRIPTION        AS description,
        e.REASONCODE         AS reason_code,
        e.REASONDESCRIPTION  AS reason_description,
        p.GENDER             AS gender,
        p.BIRTH_YEAR         AS birth_year
FROM    kardia_silver.silver_encounters e
LEFT JOIN kardia_silver.silver_patients  p
       ON e.PatientID = p.ID;